In [89]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [5]:
import gzip
import simplejson as json

In [6]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:
from statistics import mean
def describe(key):
    key_list=[script[key] for script in scripts]
    sorted_list=sorted(key_list)
    total = sum(key_list)
    avg = mean(key_list)
    s = (sum([(key-avg)**2  for key in key_list])/len(key_list))**0.5
    one_fourth=(len(sorted_list)//4)
    q25 = sorted_list[one_fourth]
    med = sorted_list[2*one_fourth]
    q75 = sorted_list[3*one_fourth]
    print(total, avg, s, q25, med, q75)

    return (total, avg, s, q25, med, q75)

In [10]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

4410054 11.522744731217633 33.11216633980368 1 3 8
316356836 826.5883059943667 3872.1810146096263 30 120 466
29048309.790000338 75.89844899484227 197.5728266277508 7.7 22.62 65.94
27053937.599999707 70.68748295124972 183.2673189530385 7.25 21.24 61.53


In [11]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [12]:
bnf_names = {script['bnf_name'] for script in scripts}
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [13]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [14]:
from collections import Counter
bnf_name_item_sum={}
for each_bnf in bnf_names:
    for each_prescription in groups[each_bnf]:
        bnf_name_item_sum[each_bnf]=bnf_name_item_sum.get(each_bnf,0)+each_prescription['items']

count=Counter(bnf_name_item_sum)
max_item = count.most_common(1)

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [15]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [16]:

def group_by_field(data, fields):
    groups = {}
    for d in data:
        key=tuple(d[f] for f in fields)
        if key in groups:
            groups[key].append(d)
        else:
            groups[key]=[d]
        
    return groups

In [17]:
from collections import Counter
groups = group_by_field(scripts, ('bnf_name',))
bnf_name_item_sum2={}
for each_group in groups:
    for each_prescription in groups[each_group]:
        bnf_name_item_sum2[each_group]=bnf_name_item_sum2.get(each_group,0)+each_prescription['items']


counter=Counter(bnf_name_item_sum2)
test_max_item = counter.most_common(1)

assert test_max_item == max_item

AssertionError: 

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [18]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = min(practice_postal[practice['code']],practice['post_code'])
    else:
        practice_postal[practice['code']] = practice['post_code']

In [19]:
len(practice_postal)

10843

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [20]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [21]:
joined = scripts[:]
print(scripts[0])
for script in joined:
    script['post_code'] = practice_postal[script['practice']]
joined[0]

{'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'N81013', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000}


{'bnf_code': '0101010G0AAABAB',
 'items': 2,
 'practice': 'N81013',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'nic': 5.98,
 'act_cost': 5.56,
 'quantity': 1000,
 'post_code': 'SK11 6JL'}

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [22]:
grouped = group_by_field(joined,('post_code',))

In [23]:
items_by_post = [(postcode[0],sum([script['items'] for script in items])) for postcode,items in grouped.items()]

In [24]:
items_by_post[0]

('SK11 6JL', 110071)

In [25]:
postal_totals = sorted(items_by_post)[:100]
grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

In [27]:
def sum_of_key(data,key):
    return sum([d[key] for d in data])

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [29]:
total_items_by_bnf_post_items = group_by_field(joined,('post_code','bnf_name'))
total_items_by_bnf_post_items[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF')]

[{'bnf_code': '0301011R0AAAPAP',
  'items': 300,
  'practice': 'M85078',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'nic': 622.5,
  'act_cost': 580.01,
  'quantity': 415,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0301011R0AAAPAP',
  'items': 149,
  'practice': 'M85774',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'nic': 264.0,
  'act_cost': 246.25,
  'quantity': 176,
  'post_code': 'B11 4BW'},
 {'bnf_code': '0301011R0AAAPAP',
  'items': 257,
  'practice': 'Y02620',
  'bnf_name': 'Salbutamol_Inha 100mcg (200 D) CFF',
  'nic': 736.5,
  'act_cost': 685.0,
  'quantity': 491,
  'post_code': 'B11 4BW'}]

In [42]:
grouped_by_bnf_post = group_by_field(joined,('post_code','bnf_name'))
total_items_by_bnf_post=[{'post_code':group[0],'bnf_name':group[1],'total':sum_of_key(value,'items')} for group,value in grouped_by_bnf_post.items()]
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [48]:
group_by_post=group_by_field(total_items_by_bnf_post,('post_code',))

In [49]:
group_by_post[('B11 4BW',)][:5]

[{'post_code': 'B11 4BW', 'bnf_name': 'Infacol_Susp 40mg/ml S/F', 'total': 10},
 {'post_code': 'B11 4BW',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'total': 15},
 {'post_code': 'B11 4BW',
  'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
  'total': 14},
 {'post_code': 'B11 4BW',
  'bnf_name': 'Gaviscon Advance_Liq (Aniseed) (Reckitt)',
  'total': 70},
 {'post_code': 'B11 4BW',
  'bnf_name': 'Gaviscon Advance_Liq (Peppermint) S/F',
  'total': 16}]

In [56]:
total_items={}
for each_group,value in group_by_post.items():
    total_items[each_group[0]]=sum_of_key(value,'total')

In [54]:
total_items[0]

{'post_code': 'SK11 6JL', 'total_items': 110071}

In [50]:
max_total_dict=[]
for each_group,value in group_by_post.items():
    post_code=each_group[0]
    bnf_name=None
    total=0
    for each in value:
        if each['total']>total:
            total=each['total']
            bnf_name=each['bnf_name']
        elif total==each['total']:
            bnf_name=min(bnf_name,each['bnf_name'])
    
    
    max_total_dict.append({'post_code':post_code,'bnf_name':bnf_name,'final_total':total})
    

In [51]:
max_total_dict[0]

{'post_code': 'SK11 6JL',
 'bnf_name': 'Omeprazole_Cap E/C 20mg',
 'final_total': 3219}

In [38]:



total_items_by_post=group_by_field(total_items,('post_code',))   
    
assert len(total_items_by_post) == 118

In [39]:
len(total_items_by_post)

118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [76]:
items_by_region=[]
for each in max_total_dict:
    items_by_region.append((each['post_code'],each['bnf_name'],each['final_total']/total_items[each['post_code']]))
    
                
items_by_region=sorted(items_by_region)[:100]          
            
        

In [77]:
len(items_by_region)

100

In [101]:
max_item_by_post[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF')]

KeyError: ('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF')

118

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [63]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [78]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*